##### Setup
Error: the datatype for boolean values in google sheet seems to be either unretrievable or (if changed to 1 or 0) converted to float which would give an error in model run, if this can be fix

In [1]:
import sys
sys.path.append('src')



#Data Analysis 
import geopandas as gpd
import pandas as pd
import numpy as np



#Plot
import matplotlib.pyplot as plt
import gc


#multiprocessing
import multiprocessing
import brun


#model
from school_model import School

#spreadsheets
import spreadsheets

# Parameter Setup
Here using a test trial small schedule with 5 steps of class per day
<br>
Run for 15 days

In [2]:
map_path = "../layouts/schoollayout1/schoollayout1.shp"
schedule_path = "small_schedule.csv" #small schedule has only 5 steps per day for testing purposes
#full day schedule should be "day_schedule.csv" 


# if you wish to use google sheet parameter input, you'll have to follow instructions for getting credentials of the sheet api
# follow the first two steps in:
# https://developers.google.com/sheets/api/quickstart/python
# and save your credentials in the src folder as listed below
SHEET_URL = 'https://docs.google.com/spreadsheets/d/1Quyyey5B_kdQK1_OU0OkIDGZE27tuUfjZ6hCsV99-sM'
credentials = './src/credentials.json'


# two types of parameter setups available for batchrunner
# pre-setup for fixed/variable parameter dictionaries (consistant with mesa batchrunner)
######################
grade_N = 350
KG_N = 50
preschool_N = 50
special_education_N = 10
faculty_N = 40
seat_dist = 12
mask_prob = 0.516
days = 15
max_steps = 15*5
iterations = 1
####################

In [4]:
# fixed parameters are model input parameters that are static for all batch running jobs
fixed_parameters = {}


fixed_parameters['grade_N'] = grade_N
fixed_parameters['KG_N'] = KG_N
fixed_parameters['preschool_N'] = preschool_N
fixed_parameters['special_education_N'] = special_education_N
fixed_parameters['faculty_N'] = faculty_N
fixed_parameters['seat_dist'] = 12
fixed_parameters['init_patient'] = 3
fixed_parameters['mask_prob'] = 0.25

# same logic as fixed parameters, only these will be batch ran for all combinations 
variable_parameters = {}
variable_parameters['attend_rate'] = [0.25, 0.5, 0.75, 1]
variable_parameters['inclass_lunch'] = [True, False]
#variable_parameters['mask_prob'] = [0, 0.25, 0.5, 0.75, 1]

In [5]:
# dataframe parameter setup that can port with google sheet
# TODO: currently the sheet script cannot retrieve boolean values so this input method is not fully functioning
# namely inclass_lunch
# should be an easy fix

param_df = spreadsheets.return_params_df(sheet_url=SHEET_URL, credentials=credentials)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=539291806604-l4elavfdt9fff15g37eon1ilbeeijm6j.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A61248%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets.readonly&state=CC6SY0f23ag7rxlPbDZgTdhALLctyH&access_type=offline


In [6]:
param_df

,grade_N,KG_N,preschool_N,special_education_N,faculty_N,seat_dist,init_patient,mask_prob,attend_rate,inclass_lunch
1,350,50,10,10,40,12,3,0.516,0.5,1
2,350,50,10,10,40,12,3,0.000,0.5,1
3,350,50,10,10,40,12,3,0.516,0.5,0
4,350,50,10,10,40,12,3,0.000,0.5,0
5,350,50,10,10,40,12,3,0.516,1.0,1
6,350,50,10,10,40,12,3,0.000,1.0,1
7,350,50,10,10,40,12,3,0.516,1.0,0
8,350,50,10,10,40,12,3,0.000,1.0,0


# Multiprocessing Run

In [6]:
if __name__ ==  '__main__': 
    # for more on multiprocessing, check src/brun 
    batchrunner = brun.BatchrunnerMP(model=School, 
                                     map_path=map_path,
                                     schedule_path=schedule_path,
                                     max_steps=max_steps, 
                                     iterations=iterations, 
                                     fixed_parameters=fixed_parameters, 
                                     variable_parameters=variable_parameters)
    results = batchrunner.run_all()

Your system has 8 available processors.


In [7]:
len(results)

8